In [1]:
import torch

In [2]:
from src.nn.train import train, load_checkpoint
from src.dataset.dataset import SparkifyDataset
from src.nn.Transformer import TransformerModel
from torch.optim.adam import Adam
from torch.utils.data import DataLoader
from src.preprocess.collate_fn import mat_collate_fn

In [3]:
checkpoint_path = "./checkpoints/sparkify_transformer"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model, opt, seen_epochs, train_ds, test_ds, writer, checkpoint_path, ignore_index, cross_entropy_weights = load_checkpoint(
    checkpoint_path,
    TransformerModel,
    Adam,
    device=device
)

In [5]:
dl_test = DataLoader(test_ds, batch_size=32, shuffle=False, collate_fn=mat_collate_fn)

In [6]:
outputs = []
targets = []
dl_iter = iter(dl_test)
for i in range(10):
    try:
        sample = next(dl_iter)
    except StopIteration:
        dl_iter = iter(dl_test)
        sample = next(dl_iter)
    emb, pos, targ, mask = sample
    emb = emb.to(device)
    pos = pos.to(device)
    mask = mask.to(device)
    targ = targ.to(device)
    output = model(emb, pos, mask)
    outputs.append(output.to("cpu"))
    targets.append(targ.to("cpu"))

/run/media/john/Secondary/Documents/Cornell/Spring 2024/ORIE 4741 Learning From Data/Project/proj/lib64/python3.11/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:177.)
  return _nested.nested_tensor(


In [7]:
torch.save(outputs, "outputs.pt")
torch.save(targets, "targets.pt")